<a href="https://colab.research.google.com/github/juliuserbach/Semantic-Features/blob/master/Object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

# install dependencies: (use cu100 because colab is on CUDA 10.0)
!pip install -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
torch.__version__
!gcc --version
# opencv is pre-installed on colab


In [0]:

# install detectron2:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu100/index.html

In [0]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [0]:
!mkdir data/Results

DATA_DIR="data/leftImg8bit/train_extra/schweinfurt"
SAVE_DIR="data/Results"

model_file="COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file(model_file))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model_file)

predictor = DefaultPredictor(cfg)

import glob
import os
import json

images_path = list(glob.iglob(os.path.join(DATA_DIR, '*.*')))
n_images = len(images_path)
i = 0
# create empty dictionary 
features = {}
# create empty list of the results
features['results'] = []

for image_path in images_path:
  # Load image from path
  image = cv2.imread(image_path)
  # Predict bounding boxes and classes
  results = predictor(image)
  print("Image number:{} of {} is beeing processed".format(i+1, n_images))

  # Save visualization in path DATA_DIR/Results with same name as images
  v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
  v = v.draw_instance_predictions(results["instances"].to("cpu"))
  head, tail = os.path.split(image_path)
  name = os.path.join(SAVE_DIR,"{}".format(tail))
  cv2.imwrite(name, v.get_image()[:, :, ::-1])

  # export classes and bounding boxes
  classes_tensor = results["instances"].pred_classes
  boxes_tensor = results["instances"].pred_boxes
  classes = classes_tensor.cpu().tolist()
  boxes = []
  box = boxes_tensor.to("cpu").tensor.tolist()
  # put them in a dicitonary to later save them into a json file
  features['results'].append({'image_id': tail, 'classes': classes, 'boxes': box})
  i +=1

In [0]:
# Write to json file
import json
# format: row: image_id , [class_ids], [boxes] , with boxes as x1, y1, x2, 
FEATURE_FILE = 'results.json'
with open(FEATURE_FILE, 'w') as outfile:
    json.dump(features, outfile)